# Tarea 2 - Pre-Procesamiento de datos
## Caso Amazon_Baby

### ITESM - Escuela de Ingeniería y Ciencias
#### Maestría en Ciencias Computacionales
#### Tópicos AI 2
##### Dr. Luis Eduardo Falcón Morales

---

## Carlos E. Hdz Rincón - A01181616

# Prerequisites and environment preparations

## Installing dependencies

Descomentamos la siguiente línea para poder instalar las dependencias de Python necesarias.

In [ ]:
#!pip install nltk pandas keras tensorflow

## Env variables

Antes de correr los imports se deben configurar las siguientes variables de entorno:

1. DATA_LOCATION. Debe ser setteada al folder donde se encuentren los archivos RAR/CSV de donde se leerán los datos
1. NLTK_DATA. (Opcional) Es una variable de entorno especial usada por NLTK que indica dónde se deben bajar los textos necesarios para su uso. Si la variable no se configura, por default, NLTK guardará sus archivos en una carpeta específica.

In [2]:
# Set this variable to the location of the files
#%env DATA_LOCATION=/content/drive/MyDrive/Colab Notebooks/TopicosAI2/Assignments
%env DATA_LOCATION=/workspace/jupyter_workspace/datasets
!echo $DATA_LOCATION

# Needed for downloading the NLTK Library Text files
#%env NLTK_DATA=/content/drive/MyDrive/Datasets/nltk/
%env NLTK_DATA=/workspace/jupyter_workspace/datasets/nltk/
!echo $NLTK_DATA

env: DATA_LOCATION=/workspace/jupyter_workspace/datasets
/workspace/jupyter_workspace/datasets
env: NLTK_DATA=/workspace/jupyter_workspace/datasets/nltk/
/workspace/jupyter_workspace/datasets/nltk/


## RAR extraction

Descomentar y ejecutar la siguiente célula si es que el archivo CSV se encuentra comprimido en un archivo RAR.

In [3]:
#!unrar x "$DATA_LOCATION/amazon_baby.rar" "$DATA_LOCATION"
#!rm "$DATA_LOCATION/amazon_baby.rar"
#!ls -lah "$DATA_LOCATION"

## NLTK File downloading

In [ ]:
import nltk

nltk.download("wordnet")

## Imports

In [4]:
import os
import re

from IPython.display import display

import pandas as pd

from keras.preprocessing.text import Tokenizer

from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer

# Connect to Google Drive if hosting the files there
#from google.colab import drive
#drive.mount('/content/drive')

## Tools

In [5]:
def print_marquee(msg: str):
    """
    Prints a centered message with a marquee of * 
    """
    marquee_width = len(msg) + 4
    print("\n")
    print("*" * marquee_width)
    print(f"* {msg} *")
    print("*" * marquee_width)

## Data loading

In [6]:
data_root_location = os.environ["DATA_LOCATION"]

data = pd.read_csv(f"{data_root_location}/amazon_baby.csv",
                   header=0)

display(data.head(5))

print_marquee("Data General information")
display(data.info())

display(data.describe())

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5




****************************
* Data General information *
****************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    183213 non-null  object
 1   review  182702 non-null  object
 2   rating  183531 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


None

,rating
count,183531.000000
mean,4.120448
std,1.285017
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


# a.

A partir del archivo `amazon_baby.csv` que se encuentra en Canvas resuelve los siguientes incisos.

Llamemos a los comentarios de la columna `review`, `review01`. Realiza un limpiado de los comentarios `review01` usando únicamente expresiones regulares Aplica las expresiones regulares que consideres adecuado para obtener un texto que posteriormente será utilizado como entrada de una red neuronal para clasificar su sentimiento.


In [7]:
review01 = data["review"].copy()

# Fill NA Review and Name values
review01 = review01.fillna("na")

# Clean up reviews using regex
def clean_up_reviews(text: str) -> str:
    
    # Convert to lowercase
    words = str(text).lower() if not isinstance(text, str) else text.lower()

    words = words.split()

    # Remove all non-alphanumeric characters
    remove_regex = r"[^a-zA-Z0-9]"

    result = []
    
    for w in words:
        w = re.sub(remove_regex, "", w)
        
        if w:
            result.append(w)
    
    result = " ".join(result)

    return result

review01 = review01.apply(clean_up_reviews)

display(review01.head(10))

print_marquee("Comparison")
print(f"Before:\n {data.iloc[0]['review']}")
print(f"After:\n {review01[0]}")

0    these flannel wipes are ok but in my opinion n...
1    it came early and was not disappointed i love ...
2    very soft and comfortable and warmer than it l...
3    this is a product well worth the purchase i ha...
4    all of my kids have cried nonstop when i tried...
5    when the binky fairy came to our house we didn...
6    lovely book its bound tightly so you may not b...
7    perfect for new parents we were able to keep t...
8    a friend of mine pinned this product on pinter...
9    this has been an easy way for my nanny to reco...
Name: review, dtype: object



**************
* Comparison *
**************
Before:
 These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.
After:
 these flannel wipes are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipesocean blue12 countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipes for hands and faces and have been usingthirsties 6 pack fab wipes boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles


# b.

Aplica los algoritmos de stemming vistos en clase al primer comentario que se tiene en `review01` (es decir, solo utiliza el comentario que inicia _"These flannel wipes are OK…"_) y elije el que consideres el mejor resultado por simple inspección visual.



In [8]:
first_review = review01[0]

def stem_text(stemmer, text: str) -> str:
    return " ".join([stemmer.stem(w) for w in text.split()])

print_marquee("Original phrase")
print(first_review)

print_marquee("PorterStemmer")
print(stem_text(PorterStemmer(), first_review))

print_marquee("Snowball Stemmer (Porter2)")
print(stem_text(SnowballStemmer("english"), first_review))

print_marquee("Lancaster Stemmer")
print(stem_text(LancasterStemmer(), first_review))



*******************
* Original phrase *
*******************
these flannel wipes are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipesocean blue12 countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipes for hands and faces and have been usingthirsties 6 pack fab wipes boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles


*****************
* PorterStemmer *
*****************
these flannel wipe are ok but in my opinion not worth keep i also order someims vims cloth wipesocean blue12 countwhich are larger had a nicer softer textur and just seem higher qualiti i use cloth wipe for hand and face and have been usingthirsti 6 pack fab wipe boyfor about 8 month now and need to replac them becaus they are start to get rough and have had stink issu for a while that strip no longer handl


***************************

---
Si combinaremos el Stemmer con un Lemmatizer, lo mejor será usar Porter Stemmer ya que éste preserva más del sentido original del texto.

Snowball y Porter sacaron el mismo resultado ya que la mayor ventaja de Snowball es que soporta diferentes lenguajes. Dado que Snowball es una versión más nueva de Porter, es Snowball el stemmer que usaremos.

# c.

Aplica ahora los algoritmos de lemmatization vistos en clase al mismo primer comentario original de `review01`. Por simple inspección, ¿cuál crees que simplifica mejor el contenido del mensaje sin perder su sentido original?

---

NLTK provée sólo 1 lemmatizer, WordNetLemmatizer que usa el corpus WordNet para realizar sus operaciones. Un lemmatizer simplifica mejor el contenido de un mensaje sin perder su sentido original pero es más costoso computacionalmente que un stemmer.


In [9]:
def lemmatize_text(lemmatizer, text: str) -> str:
    return " ".join([lemmatizer.lemmatize(w) for w in text.split()])

print_marquee("Original phrase")
print(first_review)

print_marquee("WordNet Lemmatizer")
print(lemmatize_text(WordNetLemmatizer(), first_review))



*******************
* Original phrase *
*******************
these flannel wipes are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipesocean blue12 countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipes for hands and faces and have been usingthirsties 6 pack fab wipes boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles


**********************
* WordNet Lemmatizer *
**********************
these flannel wipe are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipesocean blue12 countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipe for hand and face and have been usingthirsties 6 pack fab wipe boyfor about 8 month now and need to replace them because they are starting to get rough and have had stink issue for a while that stripping no longe

# d.

Combina los resultados de los primeros dos incisos para procesar los datos y obtener una limpieza de los comentarios.

---

Se hará primero una lemmatización del texto seguido de una stemmización, esto para que el algoritmo de lemmatización pueda reconocer palabras reales, el stemmer puede aplicar sus reglas a la salida del lemmatizer mejor que el lemmatizer a la salida del stemmer.

Veamos primero un ejemplo de cómo se ve el resultado de hacer esto.

In [10]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

print_marquee("Original phrase")
print(first_review)

print_marquee("Lemmatized and Stemmed")
print(stem_text(stemmer, lemmatize_text(lemmatizer, first_review)))



*******************
* Original phrase *
*******************
these flannel wipes are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipesocean blue12 countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipes for hands and faces and have been usingthirsties 6 pack fab wipes boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles


**************************
* Lemmatized and Stemmed *
**************************
these flannel wipe are ok but in my opinion not worth keep i also order someims vims cloth wipesocean blue12 countwhich are larger had a nicer softer textur and just seem higher qualiti i use cloth wipe for hand and face and have been usingthirsti 6 pack fab wipe boyfor about 8 month now and need to replac them becaus they are start to get rough and have had stink issu for a while that strip no longer handl


Ahora apliquemos lemmatizing y stemming a todos los reviews

In [11]:
review02 = review01.apply(lambda review: stem_text(stemmer, lemmatize_text(lemmatizer, review)))

review02.head(10)

0    these flannel wipe are ok but in my opinion no...
1    it came earli and wa not disappoint i love pla...
2    veri soft and comfort and warmer than it looks...
3    this is a product well worth the purchas i hav...
4    all of my kid have cri nonstop when i tri to w...
5    when the binki fairi came to our hous we didnt...
6    love book it bound tight so you may not be abl...
7    perfect for new parent we were abl to keep tra...
8    a friend of mine pin this product on pinterest...
9    this ha been an easi way for my nanni to recor...
Name: review, dtype: object

## i.

Utiliza el tokenizer de Keras (https://keras.io/api/preprocessing/text/ ) (lo vimos en clase) con el conjunto `review02` para generar un vocabulario con tus datos que acabas de generar. ¿De qué tamaño es tu vocabulario resultante? Llamaremos `vocabulario01` a este resultado.

---
Tamaño vocabulario01: 130324 tokens únicos.

In [12]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(review02)

In [13]:
vocabulario01 = tokenizer.word_index
display(vocabulario01)
print(f"Tamaño vocabulario: {len(vocabulario01)} tokens únicos.")

{'the': 1,
 'it': 2,
 'and': 3,
 'a': 4,
 'i': 5,
 'to': 6,
 'is': 7,
 'this': 8,
 'for': 9,
 'of': 10,
 'my': 11,
 'in': 12,
 'that': 13,
 'with': 14,
 'have': 15,
 'on': 16,
 'wa': 17,
 'but': 18,
 'we': 19,
 'so': 20,
 'use': 21,
 'not': 22,
 'babi': 23,
 'are': 24,
 'you': 25,
 'one': 26,
 'be': 27,
 'love': 28,
 'they': 29,
 'veri': 30,
 'when': 31,
 'like': 32,
 'great': 33,
 'get': 34,
 'would': 35,
 'just': 36,
 'at': 37,
 'up': 38,
 'he': 39,
 'out': 40,
 'can': 41,
 'our': 42,
 'all': 43,
 'these': 44,
 'had': 45,
 'seat': 46,
 'she': 47,
 'them': 48,
 'if': 49,
 'or': 50,
 'her': 51,
 'ha': 52,
 'littl': 53,
 'month': 54,
 'work': 55,
 'time': 56,
 'easi': 57,
 'from': 58,
 'onli': 59,
 'old': 60,
 'well': 61,
 'product': 62,
 'will': 63,
 'other': 64,
 'becaus': 65,
 'more': 66,
 'realli': 67,
 'also': 68,
 'look': 69,
 'about': 70,
 'son': 71,
 'bought': 72,
 'put': 73,
 'too': 74,
 'than': 75,
 'your': 76,
 'fit': 77,
 'go': 78,
 'no': 79,
 'much': 80,
 'good': 81,
 'dont

Tamaño vocabulario: 130324 tokens únicos.


## ii.

Aplica un filtrado sobre el tamaño de los tokens a tu `vocabulario01` para que te quedes solamente con los tokens o términos de longitud 3 o mayor. Llamaremos `vocabulario02` a este resultado. Indicar de qué tamaño es este nuevo `vocabulario02` y el porcentaje de reducción que se obtuvo con respecto a `vocabulario01`.

---
Tamaño `vocabulario02`: 129580 tokens únicos, que representa una reducción del 0.5709%

In [14]:
vocabulario02 = { token:idx for token, idx in vocabulario01.items() if len(token) >= 3 }

display(vocabulario02)

reduction_rate = 100 - ((len(vocabulario02) * 100) / len(vocabulario01))

print(f"Tamaño vocabulario02: {len(vocabulario02)} tokens únicos, que representa una reducción del {reduction_rate:.4f}%")


{'the': 1,
 'and': 3,
 'this': 8,
 'for': 9,
 'that': 13,
 'with': 14,
 'have': 15,
 'but': 18,
 'use': 21,
 'not': 22,
 'babi': 23,
 'are': 24,
 'you': 25,
 'one': 26,
 'love': 28,
 'they': 29,
 'veri': 30,
 'when': 31,
 'like': 32,
 'great': 33,
 'get': 34,
 'would': 35,
 'just': 36,
 'out': 40,
 'can': 41,
 'our': 42,
 'all': 43,
 'these': 44,
 'had': 45,
 'seat': 46,
 'she': 47,
 'them': 48,
 'her': 51,
 'littl': 53,
 'month': 54,
 'work': 55,
 'time': 56,
 'easi': 57,
 'from': 58,
 'onli': 59,
 'old': 60,
 'well': 61,
 'product': 62,
 'will': 63,
 'other': 64,
 'becaus': 65,
 'more': 66,
 'realli': 67,
 'also': 68,
 'look': 69,
 'about': 70,
 'son': 71,
 'bought': 72,
 'put': 73,
 'too': 74,
 'than': 75,
 'your': 76,
 'fit': 77,
 'much': 80,
 'good': 81,
 'dont': 82,
 'there': 83,
 'make': 84,
 'daughter': 86,
 'stroller': 88,
 'now': 89,
 'need': 90,
 'back': 92,
 'which': 93,
 'after': 94,
 'his': 95,
 'bag': 96,
 'bottl': 97,
 'even': 98,
 'purchas': 99,
 'were': 100,
 'recomme

Tamaño vocabulario02: 129580 tokens únicos, que representa una reducción del 0.5709%


## iii.

Regresa a los comentarios que obtuviste en `review02` para filtrarlos ahora con respecto a las palabras que están en `vocabulario02`. Es decir, aquellas palabras que no estén en `vocabulario02` se deberán omitir. Llamaremos a este resultado `review03`. 

In [15]:
review03 = review02.apply(lambda review: " ".join([token for token in review.split() if token in vocabulario02]))

print_marquee("Before filtering")
display(review02.head(10))

print_marquee("After filtering")
display(review03.head(10))



********************
* Before filtering *
********************


0    these flannel wipe are ok but in my opinion no...
1    it came earli and wa not disappoint i love pla...
2    veri soft and comfort and warmer than it looks...
3    this is a product well worth the purchas i hav...
4    all of my kid have cri nonstop when i tri to w...
5    when the binki fairi came to our hous we didnt...
6    love book it bound tight so you may not be abl...
7    perfect for new parent we were abl to keep tra...
8    a friend of mine pin this product on pinterest...
9    this ha been an easi way for my nanni to recor...
Name: review, dtype: object



*******************
* After filtering *
*******************


0    these flannel wipe are but opinion not worth k...
1    came earli and not disappoint love planet wise...
2    veri soft and comfort and warmer than looksfit...
3    this product well worth the purchas have not f...
4    all kid have cri nonstop when tri ween them of...
5    when the binki fairi came our hous didnt have ...
6    love book bound tight you may not abl add alot...
7    perfect for new parent were abl keep track bab...
8    friend mine pin this product pinterest decid g...
9    this been easi way for nanni record all the ke...
Name: review, dtype: object

## iv.

Obtener la matriz tfidf de `review03`, ¿de qué tamaño es? Obtener además el total de bytes consumidos por los elementos de esta matriz (puedes usar el método `nbytes` de tu objeto). Repite este resultado para cada una de las matrices _“binary”, “count”, “freq”_.

### TF-IDF matrix

In [16]:
# Refit the tokenizer to reflect changes made based on vocabulario02
tokenizer = Tokenizer()

tokenizer.fit_on_texts(review03)

In [19]:
tfidf_matrix = tokenizer.texts_to_matrix(review03, mode="tfidf")
print(f"Tamaño de la matriz TF-IDF (documentos, tokens en diccionario): {tfidf_matrix.shape}")
print(f"Bytes utilizados: {tfidf_matrix.nbytes} bytes")

Tamaño de la matriz TF-IDF (documentos, tokens en diccionario): (183531, 129581)
Bytes utilizados: 190257044088 bytes


In [27]:
tfidf_matrix[0:5]

array([[0.        , 0.        , 2.08696612, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.67841998, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.78516404, 1.35413867, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 2.31302273, 1.67841998, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.35413867, ..., 0.        , 0.        ,
        0.        ]])

### Binary Matrix

In [22]:
binary_matrix = tokenizer.texts_to_matrix(review03, mode="binary")
print(f"Tamaño de la matriz Binary (documentos, tokens en diccionario): {binary_matrix.shape}")
print(f"Bytes utilizados: {binary_matrix.nbytes} bytes")

Tamaño de la matriz Binary (documentos, tokens en diccionario): (183531, 129581)
Bytes utilizados: 190257044088 bytes


In [28]:
binary_matrix[0:5]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

### Count Matrix

In [23]:
count_matrix = tokenizer.texts_to_matrix(review03, mode="count")
print(f"Tamaño de la matriz Count (documentos, tokens en diccionario): {count_matrix.shape}")
print(f"Bytes utilizados: {count_matrix.nbytes} bytes")

Tamaño de la matriz Count (documentos, tokens en diccionario): (183531, 129581)
Bytes utilizados: 190257044088 bytes


In [29]:
count_matrix[0:5]

array([[0., 0., 5., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 1., 2., ..., 0., 0., 0.],
       [0., 7., 3., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.]])

### Freq Matrix

In [24]:
freq_matrix = tokenizer.texts_to_matrix(review03, mode="freq")
print(f"Tamaño de la matriz Freq (documentos, tokens en diccionario): {freq_matrix.shape}")
print(f"Bytes utilizados: {freq_matrix.nbytes} bytes")

Tamaño de la matriz Freq (documentos, tokens en diccionario): (183531, 129581)
Bytes utilizados: 190257044088 bytes


In [30]:
freq_matrix[0:5]

array([[0.        , 0.        , 0.07462687, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.13043478, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05      , 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.12727273, 0.05454545, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.03389831, ..., 0.        , 0.        ,
        0.        ]])